In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import gensim
from gensim.models.word2vec import LineSentence
import time
start_time = time.time()

In [2]:
import pandas as pd
df = pd.read_csv('./final_15.csv')
# df.head(1)
df_hotel = pd.read_csv('./output.csv')

print(f'大表: {df.shape}')
print(f'飯店表: {df_hotel.shape}')

大表: (12140, 5)
飯店表: (12140, 2)


In [3]:
# 統一欄位型態，後面再merge
df['aut_id'] =df['aut_id'].astype('str')
df_hotel['aut_id'] = df_hotel['aut_id'].astype('str')
df=df.merge(df_hotel,on=['aut_id'],how='inner')
# df.shape
df = df.drop_duplicates(subset='content', keep='first', ignore_index=True)
df.shape

(12011, 6)

In [24]:
# 檢查是否有空值(non)
check_for_nan = df.isnull().values.any()
print (check_for_nan)
df = df.dropna(subset=['hotel_name'])

True


In [4]:
# 轉換 list型態
df['hotel_name']=df['hotel_name'].apply(lambda x:eval(x)) # str轉 lsit
df['keywords']=df['keywords'].apply(lambda x:eval(x))
df.head(3)

TypeError: eval() arg 1 must be a string, bytes or code object

建立資料

In [6]:
# 將hotel_name多的拆分成不同的row

# def split_row(data, column):
#     row_len = list(map(len, data[column].values))
#     rows = []
#     for i in data.columns:
#         if i == column:
#             row = np.concatenate(data[i].values)
#         else:
#             row = np.repeat(data[i].values, row_len)
#         rows.append(row)
#     # return row_len
#     return pd.DataFrame(np.dstack(tuple(rows))[0], columns=data.columns)

# df = split_row(df, column='hotel_name')

In [36]:
df = df[['author', 'keywords', 'hotel_name']]
df.head(40)

,author,keywords,hotel_name
0,欣旅程欣生活,"[飯店, 旅程, 早餐, 推薦, 餐廳, 雙人, 板南線, 景點, 疫情, 朋友, 時間, ...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
1,欣旅程欣生活,"[餐廳, 感受, 感覺, 鏡子, 旅館, 椅子, 檢視, 著實, 旅程, 太太, 法皇, 物...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
2,欣旅程欣生活,"[需要, 太貼, 入住, 提供, 餐廳, 商旅, 連接, 加碼, 房間, 喜歡, 顏色, 卸...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
3,欣旅程欣生活,"[房間, 老街, 感覺, 私客, 方便, 訂房, 抽到, 紅包, 失望, 可愛, 告知, 時...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
4,欣旅程欣生活,"[飯店, 電梯, 看看, 房間, 咖啡, 昏暗, 旅程, 膠囊, 遊船, 市區, 美食, 玩...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
5,欣旅程欣生活,"[民宿, 蜻蜓, 蘑菇, 驚喜, 房間, 海景, 私房, 要來, 去看房, 熱情, 滿滿, ...","[捷絲旅台北三重館西門町店, 悠逸商旅台北士林, 綠野仙蹤]"
6,Laura,"[旅店, 成文, 新生, 提供, 飯店, 文創, 風格, 忠孝, 造型, 塗鴉, 園區, 銀...","[天成文旅華山町華山文創園區, 美好生活民宿近羅東夜市]"
7,萊恩先生,"[預訂, 入住, 方案, 早餐, 折抵, 餐廳, 訂房, 連結, 支付, 酒店, 飯店, 餐...","[台北喜來登大飯店, 大地酒店, 和逸飯店台北忠孝館, 台北亞都麗緻大飯店, 晶華酒店, 北..."
8,萊恩先生,"[預訂, 連結, 早餐, 入住, 方案, 酒店, 飯店, 雙人, 加價, 面額, 興券, 不...","[台北喜來登大飯店, 大地酒店, 和逸飯店台北忠孝館, 台北亞都麗緻大飯店, 晶華酒店, 北..."
9,維克,"[房間, 推薦, 酒店, 鳥瞰, 整理, 飯店, 老街, 房型, 價為, 空間, 參考, 訂...","[台北W酒店, 台北新板希爾頓酒店, 六福萬怡酒店, 台北萬豪酒店, 美福大飯店, 凱旋酒店..."


In [27]:
from gensim.models import Word2Vec

datas = [i for i in df['keywords']]
model = Word2Vec(datas, vector_size=200, min_count=1, workers=7, window =3, sg = 1) #size=40

In [32]:
# 找文章的斷字 
exp_sent_1 = df.iloc[1847,1]
print('1',exp_sent_1)
exp_sent_2 = df.iloc[3190,1]
print('2,',exp_sent_2)

1 ['入住', '空間', '椿通', '房間', '旅館', '地方', '提供', '街道', '整體', '手續', '風格', '想要', '寢室', '辦理', '體驗']
2, ['孩子', '帳篷', '民宿', '大樹', '大人', '遊客', '中心', '遊戲', '小朋友', '浴室', '積木', '專線', '準備', '嬰兒', '好吃']


In [33]:
# # 統一維度
vec_sent_1_w2v = np.zeros(200)
vec_sent_2_w2v = np.zeros(200)

for w in exp_sent_1:
    vec_sent_1_w2v += model.wv[w]
print(vec_sent_1_w2v)
for w in exp_sent_2:
    vec_sent_2_w2v += model.wv[w]
print(vec_sent_2_w2v)

[-6.86658744e-01 -1.41062122e+00 -1.34950040e+00  2.88700157e+00
  8.13047151e-01 -1.56136857e+00  2.86792528e+00  4.83360088e+00
  1.25056473e+00  1.01110991e-01  5.39576089e-01 -1.56245252e+00
  7.09913839e-01  2.57221404e+00 -1.62509016e+00 -2.14571367e+00
  3.91614621e-01 -1.06845105e+00  6.53474272e-01 -5.39806559e+00
  1.81864578e+00 -1.85064726e-01 -8.67928908e-01  8.84178163e-01
 -1.41337647e+00  1.53708579e-01  3.44550600e-01 -1.05903102e+00
 -1.23528058e+00  3.21109184e+00  3.58528915e+00 -3.28431950e-01
  2.63131991e+00  6.06823054e-01 -1.20022342e+00  2.68488663e+00
  8.99220618e-01 -1.07081627e+00  1.92263698e-01 -4.74113907e+00
 -2.35871320e+00 -1.55687138e-01  7.53770482e-01  3.38106320e-01
  1.28931335e+00  1.56277042e+00  2.25669056e-01 -2.22723163e+00
  1.67140156e+00  7.90057694e-01  1.00171469e+00 -1.79432832e+00
 -2.48982173e+00 -7.82536828e-01  2.24386381e+00 -8.02707149e-01
  2.69898145e-01 -2.47959261e+00 -3.38664092e+00  5.57345827e-02
 -3.63773013e+00  8.20103

In [34]:
# 兩篇文章的相似度
w2v_similarity = np.dot(vec_sent_1_w2v, vec_sent_2_w2v) / (np.linalg.norm(vec_sent_1_w2v)*np.linalg.norm(vec_sent_2_w2v))
print('similarity of Word2Vec+: ', w2v_similarity)

similarity of Word2Vec+:  0.9944686008774649


In [12]:
from operator import itemgetter

for i in range(1,10):
    similar=[]
    for j in range(len(df)):
        exp_sent_1 = df.iloc[i,1]
        exp_sent_2 = df.iloc[j,1]
        vec_sent_1_w2v = np.zeros(30)
        vec_sent_2_w2v = np.zeros(30)
        for w in exp_sent_1:
            vec_sent_1_w2v += model.wv[w]
        for w in exp_sent_2:
            vec_sent_2_w2v += model.wv[w]
        w2v_similarity = np.dot(vec_sent_1_w2v, vec_sent_2_w2v) / (np.linalg.norm(vec_sent_1_w2v)*np.linalg.norm(vec_sent_2_w2v))
        similar.append((df.iloc[j,0],w2v_similarity))
        similar.sort(key=itemgetter(1), reverse=True)    
# 找出 top3 的 飯店
    kv={"hotel":df.iloc[i,0],"list":[similar[1][0],similar[2][0],similar[3][0]]}
    print(kv)

# Test

In [28]:
a = ['親子', '免治', '晚餐', '開車', '夜店']

In [38]:
similar=[]
vec_sent_1_w2v = np.zeros(200)
vec_sent_2_w2v = np.zeros(200)
for i in range(5000):
    exp_sent_1 = a
    exp_sent_2 = df.iloc[i,1]
    for w in exp_sent_1:
        vec_sent_1_w2v += model.wv[w]
    for w in exp_sent_2:
        vec_sent_2_w2v += model.wv[w]
    w2v_similarity = np.dot(vec_sent_1_w2v, vec_sent_2_w2v) / (np.linalg.norm(vec_sent_1_w2v)*np.linalg.norm(vec_sent_2_w2v))
    similar.append((df.iloc[i,2],w2v_similarity))
    similar.sort(key=itemgetter(1), reverse=True) 
print(similar[:10])
# # 找出 top3 的 飯店
#     kv={"list":[similar[1][0],similar[2][0],similar[3][0]]}
#     print(kv)
        
    

[(['捷絲旅台北三重館西門町店', '悠逸商旅台北士林', '綠野仙蹤'], 0.9964356096089925), (['捷絲旅台北三重館西門町店', '悠逸商旅台北士林', '綠野仙蹤'], 0.9961399108352225), (['天成文旅華山町華山文創園區', '美好生活民宿近羅東夜市'], 0.9960510171831809), (['捷絲旅台北三重館西門町店', '悠逸商旅台北士林', '綠野仙蹤'], 0.9954255467734899), (['捷絲旅台北三重館西門町店', '悠逸商旅台北士林', '綠野仙蹤'], 0.9953646244835145), (['捷絲旅台北三重館西門町店', '悠逸商旅台北士林', '綠野仙蹤'], 0.9949718421666478), (['台北喜來登大飯店', '大地酒店', '和逸飯店台北忠孝館', '台北亞都麗緻大飯店', '晶華酒店', '北投麗禧溫泉酒店', '台北新板希爾頓酒店', '台北士林萬麗酒店', '台北萬豪酒店', '捷絲旅台北三重館', '台北君悅酒店', '北投老爺酒店', '台北遠東香格里拉', '台北文華東方酒店'], 0.9948389234956797), (['台北喜來登大飯店', '大地酒店', '和逸飯店台北忠孝館', '台北亞都麗緻大飯店', '晶華酒店', '北投麗禧溫泉酒店', '台北新板希爾頓酒店', '台北士林萬麗酒店', '台北萬豪酒店', '捷絲旅台北三重館', '台北君悅酒店', '北投老爺酒店', '台北遠東香格里拉', '台北文華東方酒店'], 0.9944129888741947), (['台北W酒店', '台北新板希爾頓酒店', '六福萬怡酒店', '台北萬豪酒店', '美福大飯店', '凱旋酒店', '台北君悅酒店', '晶華酒店'], 0.9933632834095313), (['台北W酒店', '台北新板希爾頓酒店', '六福萬怡酒店', '台北萬豪酒店', '美福大飯店', '凱旋酒店', '台北君悅酒店', '晶華酒店'], 0.9927006884813437)]


In [15]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.45724368095398 seconds ---


In [16]:
#訓練
model = Word2Vec(datas, vector_size=100, min_count=1, workers=7, window =3, sg = 1)
# model.save('w2v_model_tpk30')

編寫文本轉向量方法，將一段文字的向量用各詞向量平均值來表示，爲10維向量

In [17]:
def buildWordVector(article_w2v, text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        #print word
        try:
            vec += article_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            print (word)
            continue
    if count != 0:
        vec /= count
    return vec

In [18]:
print(type(df['keywords'][0]))

<class 'list'>


In [19]:
result = buildWordVector(model, df.loc[0]['keywords'] , 100)
print(result)
for idx in range (1, len(df)):
    result = np.concatenate((result, buildWordVector(model, df.loc[idx]['keywords'] , 100)), axis=0)

[[-0.10088898  0.05052548  0.04142478  0.03434817  0.14168762 -0.2104896
   0.2038812   0.47625036 -0.14165188 -0.06486036 -0.16187382 -0.28307417
  -0.07511755  0.17302351 -0.029239   -0.04213486  0.09443296 -0.12436204
  -0.09700787 -0.49786513  0.2199745  -0.03888056  0.00583309 -0.17004557
  -0.10669614 -0.00520497 -0.13647209 -0.27408984 -0.17905607  0.06383808
   0.29614802  0.0704757   0.2477123  -0.36811187 -0.11742757  0.21410073
   0.04827665  0.04042902 -0.10447026 -0.30829697  0.11863513 -0.11696611
   0.10683619  0.18090367  0.14404647 -0.13451099 -0.00486108 -0.1394628
   0.11951706  0.08003428 -0.00995506 -0.15658506 -0.05103858 -0.06266855
  -0.11520211  0.15675703  0.03637081 -0.06687669 -0.12128536  0.04028312
   0.05769142 -0.09474522  0.15816237 -0.05746772 -0.15571381  0.16032695
   0.13735696  0.3884145  -0.27819953  0.12901163  0.09177132 -0.02009259
   0.2301075   0.1859385   0.35090522 -0.04468488  0.22443265  0.07632988
  -0.14182072  0.10534109 -0.13459131  0

KeyError: 152

In [ ]:
#把series 轉換成dataframe格式，並且將五十維的特徵都賦值
vectors = pd.DataFrame(result, columns = ["x1","x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10","x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20", "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30", "x31", "x32", "x33", "x34", "x35", "x36", "x37", "x38" ,"x39", "x40", "x41", "x42", "x43" ,"x44", "x45", "x46" ,"x47", "x48", "x49", "x50",
                                         "x51","x52", "x53", "x54", "x55", "x56", "x57", "x58", "x59", "x60","x61", "x62", "x63", "x64", "x65", "x66", "x67", "x68", "x69", "x70", "x71", "x72", "x73", "x74", "x75", "x76", "x77", "x78", "x79", "x80", "x81", "x82", "x83", "x84", "x85", "x86", "x87", "x88" ,"x89", "x90", "x91", "x92", "x93" ,"x94", "x95", "x96" ,"x97", "x98", "x99", "x100"])
vectors.head(1)

In [ ]:
#合併dataframe
data = pd.concat([df, vectors], axis = 1)
data.head(1)
# print(data.shape)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
dt= data.drop(['bnb_art_id', 'content', 'aut_id', 'keywords', 'author'],axis=1)

In [ ]:
SSE =[]
for k in range(1,10):
    model_k = KMeans(n_clusters=k)
    model_k.fit(dt)
    SSE.append(model_k.inertia_)
X = range(1,10)
plt.xlabel('k')  
plt.ylabel('SSE')  
plt.plot(X,SSE,'o-')  
plt.show()

In [ ]:
#按6類分
kms6=KMeans(n_clusters=6)
y6=kms6.fit_predict(dt)
# print(y5)
#按5類分
kms5=KMeans(n_clusters=5)
y5=kms5.fit_predict(dt)
# print(y5)
y5=pd.DataFrame(y5, columns = ['kind-5'])
# #按4類分
kms4=KMeans(n_clusters=4)
y4=kms4.fit_predict(dt)
y4 = pd.DataFrame(y4, columns = ['kind-4'])
# 按3類分
kms3=KMeans(n_clusters=3)
y3=kms3.fit_predict(dt)
y3 = pd.DataFrame(y3, columns = ['kind-3'])

計算所有樣本的平均輪廓係數

In [ ]:
from sklearn.metrics import silhouette_score
score_3 = silhouette_score(dt, y3)
print("score = ",score_3)
score_4 = silhouette_score(dt, y4)
print("score = ",score_4)
score_5 = silhouette_score(dt, y5)
print("score = ",score_5)
score_6 = silhouette_score(dt, y6)
print("score = ",score_6)

In [ ]:
data = pd.concat([df, y4], axis = 1)
data = pd.concat([data, y5], axis = 1)
data = pd.concat([data, y3], axis = 1)

In [ ]:
data.head(5)

In [ ]:
print(data.loc[data['kind-4']==0].shape)
print(data.loc[data['kind-4']==1].shape)
print(data.loc[data['kind-4']==2].shape)
print(data.loc[data['kind-4']==3].shape)

In [ ]:
print(data.loc[data['kind-3']==0].shape)
print(data.loc[data['kind-3']==1].shape)
print(data.loc[data['kind-3']==2].shape)

In [ ]:
print(data.loc[data['kind-5']==0].shape)
print(data.loc[data['kind-5']==1].shape)
print(data.loc[data['kind-5']==2].shape)
print(data.loc[data['kind-5']==3].shape)
print(data.loc[data['kind-5']==4].shape)

In [ ]:
# data = data[['author', 'kind-4', 'kind-5']]
# data.to_csv('./check.csv')

# Test

In [ ]:
a = ['早餐', '日式', '親子', '友善', '安靜']
b = buildWordVector(model, a , 50)
print(b)
# y4 = pd.DataFrame(y4, columns = ['kind-4'])